In [1]:
from validation.models import RandomForest,Utils

In [2]:
from validation.data import Modeling

In [3]:
from validation.data import Files

## Importando o datasset:

Utiliza a função de csv to json da classe de manipulação de arquivos criada

In [4]:
fileManipulation = Files("from_csv",["Datassets/tested.csv","Datassets/titanic.json","PassengerId"])

In [5]:
datasset = fileManipulation.jsonFile

Extrai os dados do json:

In [6]:
datasset = [datasset[i] for i in datasset]

## Cria a instância dos dados pré processados

Utilizo a classe Modeling para pré processar os dados e usar a função drops para já eliminar as colunas que não server para análise

In [7]:
titatic_data = Modeling(datasset,['Survived'],drops=["Cabin","Name","Embarked","Ticket","PassengerId"]) # Retirando colunas que não servem

In [8]:
titatic_data.encoded_features

,Pclass,Sex,Age,SibSp,Parch,Fare
0,2,1,40,0,0,140
1,2,0,56,1,0,121
2,1,1,72,0,0,167
3,2,1,29,0,0,157
4,2,0,22,1,1,8
...,...,...,...,...,...,...
413,2,1,0,0,0,154
414,0,0,47,0,0,5
415,2,1,46,0,0,125
416,2,1,0,0,0,154


# Cria a instância do modelo e realiza as previsões:

In [9]:
MyModel = RandomForest(titatic_data.X_train,titatic_data.y_train,titatic_data.X_test,titatic_data.y_test)

Fitting 3 folds for each of 1620 candidates, totalling 4860 fits


d:\ML-Hub\Ambiente-de-desenvolvimento\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
1620 fits failed out of a total of 4860.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
749 fits failed with the following error:
Traceback (most recent call last):
  File "d:\ML-Hub\Ambiente-de-desenvolvimento\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\ML-Hub\Ambiente-de-desenvolvimento\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "d:\ML-Hub\Ambiente-de-desenvolvimento\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constra

In [10]:
MyModel.gridSearchObj

{'y_pred': array(['0', '1', '0', '0', '1', '0', '1', '0', '1', '0', '0', '1', '1',
        '1', '0', '0', '1', '0', '0', '0', '0', '0', '1', '0', '1', '1',
        '1', '0', '0', '0', '0', '1', '1', '0', '1', '0', '1', '0', '1',
        '0', '1', '1', '0', '0', '0', '0', '1', '1', '1', '0', '0', '1',
        '1', '0', '0', '1', '0', '0', '1', '0', '0', '0', '1', '0', '1',
        '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '1',
        '1', '0', '0', '1', '0', '1'], dtype=object),
 'grid': GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
              param_grid={'bootstrap': [True, False],
                          'max_depth': [None, 10, 20, 30, 40, 50],
                          'max_features': ['auto', 'sqrt', 'log2'],
                          'min_samples_leaf': [1, 2, 4],
                          'min_samples_split': [2, 5, 10],
                          'n_estimators': [100, 200, 300, 400, 500]},
              verbose=2),
 'best_

## Avaliando o modelo em cima de novos dados:

In [11]:
import pandas as pd

Supondo duas pessoas novas, sendo a primeira do sexo masculino, 100 anos de idade, 6 irmãos, 3° calsse e etc... e a segunda sexo feminino, 22 anos e 1 irmão e 1° classe, no nosso exemplo a primeira morreu (0) e a segunda sobreviveu (1), agora vejamos o que o modelo nos diz disso

In [12]:
X_new = pd.DataFrame({
    'Pclass': [3, 1],
    'Sex': [1, 0], #1=male 0=female
    'Age': [100, 22],
    'SibSp': [6, 1],
    'Parch': [1, 0],
    'Fare': [15, 70]
})

In [13]:
y_new = ['0', '1']

In [14]:
X_new

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,1,100,6,1,15
1,1,0,22,1,0,70


Utilizo o melhor estimator retornado pelo meu grid search e digo para realizar uma nova previsão, utilizando minhas features criadas acima:

In [15]:
new_predictions = MyModel.gridSearchObj["best_estimator"].predict(X_new)

A baixo temos o resultado das previsões:

In [16]:
new_predictions

array(['0', '1'], dtype=object)

O modelo nos trouxe um resultado esperado, mas... vamos usar uma visualização melhor de metricas:

In [17]:
myMetrics = MyModel.acurracy_metrics(MyModel.gridSearchObj["best_estimator"],new_predictions,y_new)

In [18]:
myMetrics

{'TrainAccuracy': 1.0,
 'TestAccuracy': 1.0,
 'obs': 'O modelo não parece estar sofrendo de overfitting.'}

Pronto, agora temos um exemplo de como aplicar a abstração do modelo de random forest de forma mais simples, utilzando as classes de pré processamento e de manipulação de arquivos

Agora, realizemos um teste para exportar o modelo que criamos e treinamos:

In [19]:
export =  Utils(MyModel.gridSearchObj["best_estimator"],"dump",["titanic_random_forest_model.joblib"])

A função de dump do modulo Utils, retorna True para a variável instance, já a função load retorna o próprio modelo

In [21]:
export.instance

True